## Carrengado os dados

In [7]:
url ='https://github.com/allanspadini/curso-tensorflow-proxima-palavra/raw/main/dados/train.zip'

In [8]:
import pandas as pd

df = pd.read_csv(url, header=None, names=["ClassIndex", "Título", "Descrição"])
df.head()

,ClassIndex,Título,Descrição
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


## Pré-processamento

In [9]:
df["Texto"] = df["Título"] + " " + df["Descrição"]
df.head()

,ClassIndex,Título,Descrição,Texto
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."


O campo "ClassIndex" possui valores de 1 até 4. É importante que os dados de índice comecem em zero para que o TensorFlow possa trabalhar com eles de forma eficiente. O TensorFlow espera que os dados de entrada sejam representados como números inteiros, e esses números devem começar do zero.

In [10]:
df["ClassIndex"].unique()

array([3, 4, 2, 1])

In [11]:
df["ClassIndex"] = df["ClassIndex"] - 1
df["ClassIndex"].unique()

array([2, 3, 1, 0])

## Separando os dados em treino e teste

In [13]:
from sklearn.model_selection import train_test_split

X = df["Texto"].values
y = df["ClassIndex"].values

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=4256)